# **Analisis de sentimientos de comentarios de películas**

### Integrantes:
* **Ángela Vargas:** Desempeñó el rol de Líder de analítica al encargarse de probar los modelos y determinar cual es el mejor según las restricciones del cliente 
* **Juan Martin Santos:** Desempeñó el rol de lider de negocio asumiendo la responsabilidad de identificar la estrategia utilizada para alcanzar el objetivo del negocio.
* **Daniel Osorio:** Desempeño el rol de Líder de proyecto definiendo las fechas de reuniones y asignando los tareas equitativamente entre los integrantes. También desempñó el rol de lider de datos al haber realizado la limpieza de los datos y haberla dejado disponible para todos los integrantes.

## **Caso**

| Pregunta de entendimiento | Justificación |
| --- | --- |
| **¿Cuáles son los objetivos del negocio?** | En este caso asumimos que nuestro cliente es una página de streaming que desea tomar las reviews de las películas de su plataforma para y determinar si esta debe reemplazarse por una nueva película que pueda generar un mejor sentimiento a la audiencia o no. El sentimiento viene dado por la cantidad de buenas y malas reviews que tenga, entre mas reviews negativas tenga, peor es el sentimiento de la película. |
| **¿Cuáles son los criterios de éxito?** | Nuestro principal criterio de éxito es la precisión, ya que determinar erroneamente el sentimiento de una película puede afectar al negocio en la toma de decisiones sobre el estado de una película dentro de su plataforma. |
| **¿Cuál es la Oportunidad / problema del negocio?** | Al usar aprendizaje automático, el negocio se puede beneificar al ser más eficiente en la forma en que determinar si una película debe quedarse o no, haciendo que el público en general esté más satisfecho con el contenido que consume aumentando la fidelidad y posibles ganancias. |
| **¿Cuál es el Enfoque Analítico?** | Desde el punto de vista del aprendizaje automático, el problema se enfoca en algoritmos de clasificación en el análisis de textos. De esta forma, se etá realizando el análisis de sentimientos de películas. Este proyecto tiene comentarios de películas en español, que deben ser clasificadas en las categorías de positivo, negativo. Esto por medio de técnicas para el procesamiento de lenguaje natural. |
| **¿Quienes estan siendo beneficiados?** | El principal beneficiado a parte del negocio son los clientes que van a poder determinar tener más decisión sobre el catálogo que ofrece la empresa de streaming |
| **¿Cuáles son las técnicas y algoritmos a utilizar?** | Haremos uso de Multinomial Naives Bayes, Random forest y regresión logística |

## **Entendimiento de los datos**
Primero vamos a cargar los datos en un DataFrame para luego revisar la calidad de estos en cuanto a completitud, consistencia y unicidad.

In [1]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import nltk
# Se descargan las stopwords (despues se usan para eliminar palabras que no aportan informacion)
nltk.download('stopwords')
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from langdetect import detect
from cleanText import cleanText

# Este comando es requerido para que las visualizaciones se muestren en este notebook
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
pd.set_option('display.max_columns', None) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar

In [4]:
# Lectura de datos en formato CSV
# Los datos son almacenados en memoria usando una estructura de datos de Pandas conocida como dataframe
df_reviews = pd.read_csv('./data/MovieReviews.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/MovieReviews.csv'

In [ ]:
# Muestra el numero de filas y columnas del dataframe
df_reviews.shape

In [ ]:
# Muestra de 5 filas aleatorias del dataframe
df_reviews.sample(5)

In [ ]:
# Muestra el tipo de datos de cada columna
df_reviews.dtypes

In [ ]:
# Muestra los diferentes valores de la columna sentimiento
df_reviews['sentimiento'].value_counts()

Como podemos observar, tenemos 5000 registros (reviews) de los cuales la mitad corresponden a reviews positivas y la otra mitad a reviews negativas. Existe consistencia en los valores de la columna sentimiento. Como existen 2500 positivos y 2500 negativos, podemos decir que la completitud de los datos para la columna sentimiento es del 100%, vamos a revisar ahora la completitud para la columna *review_es*.

In [ ]:
# Muestra la cantidad de valores nulos para la columna review_es
df_reviews['review_es'].isnull().sum()

Ahora, verificaremos la unicidad de los valores de la columnas "review_es". Este paso es importante ya que al identificar registros duplicados, podemos tomar medidas que nos ayudan a mejorar la precision y eficiencia de los modelos que realizaremos.

In [ ]:
df_reviews['review_es'].duplicated().sum()

Por ultimo, verificaremos el idioma en el que se encuentran los comentarios.

In [ ]:
# Primero creamos el diccionario de los idiomas en los que estan los comentarios
langs = {}

# Luego iteramos sobre cada comentario y determinamos su idioma
for index, row in df_reviews.iterrows():
    if detect(row['review_es']) not in langs:
        langs[detect(row['review_es'])] = 1
    else:
        langs[detect(row['review_es'])] += 1

print(langs)

Como podemos observar, tenemos una completitud del 100% en la columna review_es, por lo que no es necesario realizar ningún tipo de limpieza de datos a nivel de intregridad y completitud de los datos. Sin embargo, nos percatamos que dentro del conjunto de datos hay comentarios que no estan en español y que hay comentarios repetidos, por lo que obtaremos por eliminar estas filas, esta decision se toma debido a las necesidades del negocio de analisar comentarios en español.
## **Preprocesamiento**


Primero comenzaremos por cambiar las variables de sentimiento que son positivo y negativo por 0 (siendo negativo) y 1 (siendo positivo)

In [ ]:
df_reviews['sentimiento'].replace({'negativo': 0, 'positivo': 1}, inplace=True)
df_reviews['sentimiento'].value_counts()

Para encargarnos de los comentarios repetidos obtamos por eliminar los registros. Estos no significan una gran parte de los datos por lo que quitarlos nos ayudara en el mejor funcionamiento del algoritmo

In [ ]:
df_reviews['review_es'].drop_duplicates(inplace=True)

Como libreria para limpiar las filas con los comentarios en ingles usaremos langdetect que nos permitira tomar todo el comentario y determinar el idioma en el que esta.

In [ ]:
#Iteramos en cada una de las filas del dataframe
for index, row in df_reviews.iterrows():
    # Si el valor de la fila es en la elimina
    if detect(row['review_es']) != 'es':
        df_reviews.drop(index, inplace=True)

Como libreria para el procesamiento de lenguaje natural vamos a utilizar NLTK (Natural Language Tool Kit). Primero definimos un set de stopwords en español. Las stopwords son esas palabras que no aportan significado al texto para la tarea de clasificación. Estas palabras vacías pueden ser conectores propios del idioma, pronombres o puntos y comas. Algunas stopwords del español son por ejemplo las siguientes.

In [2]:
# Seteamos las stopwords en español
stop_words = set(stopwords.words('spanish'))
# Agregamos a las stopwords tambien los signos de puntuacion
# Se agrega una u a la izquierda para indicar que es un string unicode, sin embargo no es necesario
stop_words = list(stop_words)
stop_words.extend([u'.', u'[', ']', u',', u';', u'', u')', u'),', u' ', u'('])
print(stop_words)

['tengan', 'habido', 'estada', 'estarás', 'esa', 'no', 'sentido', 'estemos', 'estuve', 'tendrían', 'hubierais', 'mis', 'estará', 'yo', 'fueses', 'estuvieran', 'fuesen', 'qué', 'quienes', 'éramos', 'donde', 'hubieses', 'teníais', 'estuvierais', 'tened', 'será', 'eso', 'habrían', 'tuvisteis', 'le', 'pero', 'otro', 'ti', 'estaríais', 'tuvo', 'mío', 'hubisteis', 'tendrás', 'antes', 'soy', 'teníamos', 'sentidos', 'estoy', 'hemos', 'para', 'tuyo', 'a', 'hubieron', 'suyo', 'fuimos', 'estado', 'su', 'también', 'unos', 'esto', 'tenidas', 'tenida', 'hubiste', 'habiendo', 'tuvieses', 'nosotras', 'estuvisteis', 'eres', 'como', 'tengas', 'somos', 'habré', 'tienen', 'tuvieran', 'ya', 'míos', 'cuando', 'hubieras', 'habríais', 'o', 'estás', 'suya', 'estarán', 'estaba', 'tendremos', 'habríamos', 'estuvieron', 'fueras', 'habida', 'es', 'poco', 'has', 'tanto', 'con', 'tenido', 'habría', 'fueran', 'esas', 'estáis', 'tuyos', 'sin', 'fuisteis', 'tendréis', 'porque', 'ese', 'serían', 'algunos', 'tuvieseis', 

Luego se utiliza el Snowball stemming algorithm, este algoritmo lo que hace es transformar/reducir las palabras a su forma base, aqui hacemos un ejemplo con la palabra 'corriendo' y 'correr'

In [3]:
snowball_stemmer = SnowballStemmer('spanish')
snowball_stemmer.stem('corriendo')

'corr'

In [ ]:
snowball_stemmer.stem('correr')

Para tokenizar utilizamos la función *wordpunct_tokenize()* que tiene la libreria NLTK. Esta función, a diferencia de la función *word_tokenize()*, almacena los signos de puntuación como tokens diferentes. El *word.isapha()* es para revisar que la palabra no sea de una sola letra como por ejemplo 'y'. Luego se revisa que la palabra en minuscula no esté entre las stopwords antes de tokenizarla. A continuación un ejemplo de como funciona

In [ ]:
# Ejemplo de como se tokenizan las palabras de una reseña
review_example = "Este es un ejemplo de una reseña en español."
review_tokens = [word.lower() for word in wordpunct_tokenize(review_example) if word.isalpha() and word.lower() not in stop_words]
print(review_tokens)

Ahora aplicamos esto para cada fila de la columna 'review_es'. Para esto definimos la funcion lamba a cada fila de la columna review utilizando el metodo apply. Se hace todo el proceso de tokenizacion mostrado anteriormente y se vuelve a juntar el texto con el metodo join(). El string resultante es ahora el nuevo valor para la fila en la columna 'review_es'.

In [ ]:
df_reviews['review_es'] = df_reviews['review_es'].apply(lambda x: ' '.join([snowball_stemmer.stem(word.lower()) for word in wordpunct_tokenize(x) if (word.isalpha() and word.lower() not in stop_words)]))

In [ ]:
df_reviews['review_es'].head(5)

Usamos el *CountVectorizer()* que nos da la cuenta de cada de cada palabra para cada review, lo cual podemos usar para los modelos que crearemos a continuación.

In [ ]:
# Extraer la cuenta de cada palabra para cada review
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_reviews['review_es'])
y = df_reviews['sentimiento']

In [ ]:
# Dividimos los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## **Modelos**

### **Multinomial Naive Bayes**

Escogimos este algoritmo porque trabaja muy bien con datos discretos, lo cual lo hace especialmente bueno para una tarea de clasificacion como si el comentario es positivo o negativo. El algoritmo realiza el calculo de la probabilidad de que cada palabra de una review pertenezca a una clase (positivo o negativo) y luego utiliza estas probabilidades con el teorema de Bayes para calcular la probabilidad de que una review pertenezca a una clase. El algoritmo asume que la probabilidad de una palabra es independiente de las otras, por lo cual es llamado 'naive' Bayes.

In [ ]:
# Entrenar el modelo
multinomialNB = MultinomialNB()
multinomialNB.fit(X_train, y_train)

# Predicción
y_train_pred = multinomialNB.predict(X_train)
y_pred = multinomialNB.predict(X_test)

# Metricas de evaluacion
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test, y_pred, pos_label=1)))
print("Precisión: {}".format(precision_score(y_test, y_pred, pos_label=1)))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred, pos_label=1)))

In [ ]:
# Matriz de confusion
cm = confusion_matrix(y_test, y_pred)

# Crear una instancia de ConfusionMatrixDisplay con la matriz de confusión
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

# Mostrar la matriz de confusión utilizando la función plot() de ConfusionMatrixDisplay
cm_display.plot(cmap='Reds')

# Añadir etiquetas y títulos
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Mostrar la matriz de confusión
plt.show()

### Random forest
Escogimos este algoritmo porque este es capaz de manejar caracteristicas de alta dimension lo que favorece en el analisis de texto por la extension de los mismos. Ademas, maneja muy bien el overfitting. El algoritmo se basa en arboles de decision y funciona de la siguiente forma: Selecciona aleatoriamente un subconjunto de muestras; Selecciona aleatoriamente un subconjunto de caracteristicas; con las muestras y caracteristicas seleccionadas se construyen varios arboles de decision; por ultimo se selecciona el arbol de decision que obtiene mayor votacion dados sus resultados.

In [ ]:
# Creamos el clasificador de random forest con 100 arboles
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
# Entrenamos el modelo
y_train_pred_rf = rf.fit(X_train, y_train)
# Realizamos la predicción
y_pred_rf = rf.predict(X_test)

# Metricas de evaluacion
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_rf))
print("Recall: {}".format(recall_score(y_test, y_pred_rf, pos_label=1)))
print("Precisión: {}".format(precision_score(y_test, y_pred_rf, pos_label=1)))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred_rf, pos_label=1)))

In [ ]:
# Matriz de confusion
cm_rf = confusion_matrix(y_test, y_pred_rf)

# Crear una instancia de ConfusionMatrixDisplay con la matriz de confusión
cm_rf_display = ConfusionMatrixDisplay(confusion_matrix=cm_rf)

# Mostrar la matriz de confusión utilizando la función plot() de ConfusionMatrixDisplay
cm_rf_display.plot(cmap='Reds')

# Añadir etiquetas y títulos
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Mostrar la matriz de confusión
plt.show()

### Regresión logística
Escogimos este algoritmo porque la regresión logística es particularmente adecuada para problemas de clasificación binaria, es decir, cuando solo hay dos categorías posibles (en este caso, positivo o negativo). En estos casos, la regresión logística puede ser más eficiente y precisa que otros algoritmos. Los coeficientes del modelo pueden utilizarse para determinar qué palabras o características tienen más influencia en la clasificación de los comentarios como positivos o negativos. Además puede escalar bien con grandes conjuntos de datos y se puede mejorar mediante técnicas avanzadas de optimización.

In [ ]:
# Crear un modelo de regresión logística y ajustarlo a los datos de entrenamiento
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Hacer predicciones en los datos de prueba y calcular la precisión
y_pred_lr = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr)

print("Precisión de la regresión logística:", accuracy)
# Metricas de evaluacion
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_lr))
print("Recall: {}".format(recall_score(y_test, y_pred_lr, pos_label=1)))
print("Precisión: {}".format(precision_score(y_test, y_pred_lr, pos_label=1)))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred_lr, pos_label=1)))

In [ ]:
# Matriz de confusion
cm_lr = confusion_matrix(y_test, y_pred_lr)

# Crear una instancia de ConfusionMatrixDisplay con la matriz de confusión
cm_lr_display = ConfusionMatrixDisplay(confusion_matrix=cm_lr)

# Mostrar la matriz de confusión utilizando la función plot() de ConfusionMatrixDisplay
cm_lr_display.plot(cmap='Reds')

# Añadir etiquetas y títulos
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Mostrar la matriz de confusión
plt.show()

#### Regresion logistica con mas iteraciones
Dado que se obtuvo un warning que decia que no se alcanzo la convergencia se sugeria aumentar el numero maximo de iteraciones, el que esta por defecto es 100 por lo tanto se decide establecer un valor mayor como 1000.

In [ ]:
# Crear un modelo de regresión logística y ajustarlo a los datos de entrenamiento
lr_1 = LogisticRegression(max_iter=1000)
lr_1.fit(X_train, y_train)

# Hacer predicciones en los datos de prueba y calcular la precisión
y_pred_lr_1 = lr_1.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr_1)

print("Precisión de la regresión logística:", accuracy)
# Metricas de evaluacion
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_lr_1))
print("Recall: {}".format(recall_score(y_test, y_pred_lr_1, pos_label=1)))
print("Precisión: {}".format(precision_score(y_test, y_pred_lr_1, pos_label=1)))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred_lr_1, pos_label=1)))

In [ ]:
# Matriz de confusion
cm_lr_1 = confusion_matrix(y_test, y_pred_lr_1)

# Crear una instancia de ConfusionMatrixDisplay con la matriz de confusión
cm_lr_1_display = ConfusionMatrixDisplay(confusion_matrix=cm_lr_1)

# Mostrar la matriz de confusión utilizando la función plot() de ConfusionMatrixDisplay
cm_lr_1_display.plot(cmap='Reds')

# Añadir etiquetas y títulos
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Mostrar la matriz de confusión
plt.show()

#### Regresion logistica con otro algoritmo de optimizacion
Dado que aumentar el numero maximo de iteraciones no mejoro el modelo se decidio probar con otro algoritmo de optimizacion. El algoritmo utilizado por defecto es 'lbfgs' (este utiliza el método de optimización de cuasi-Newton Broyden-Fletcher-Goldfarb-Shanno (BFGS)) para optimizar los parámetros del modelo). Se decide intentar con saga que utiliza una versión mejorada del método de promedio de gradientes estocásticos que admite regularización elástica y regularización L1.

In [ ]:
# Crear un modelo de regresión logística y ajustarlo a los datos de entrenamiento
lr_2 = LogisticRegression(max_iter=1000,solver='saga')
lr_2.fit(X_train, y_train)

# Hacer predicciones en los datos de prueba y calcular la precisión
y_pred_lr_2 = lr_2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr_2)

print("Precisión de la regresión logística:", accuracy)
# Metricas de evaluacion
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_lr_2))
print("Recall: {}".format(recall_score(y_test, y_pred_lr_2, pos_label=1)))
print("Precisión: {}".format(precision_score(y_test, y_pred_lr_2, pos_label=1)))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred_lr_2, pos_label=1)))

In [ ]:
# Matriz de confusion
cm_lr_2 = confusion_matrix(y_test, y_pred_lr_2)

# Crear una instancia de ConfusionMatrixDisplay con la matriz de confusión
cm_lr_2_display = ConfusionMatrixDisplay(confusion_matrix=cm_lr_2)

# Mostrar la matriz de confusión utilizando la función plot() de ConfusionMatrixDisplay
cm_lr_2_display.plot(cmap='Reds')

# Añadir etiquetas y títulos
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Mostrar la matriz de confusión
plt.show()

In [ ]:
# Crear un modelo de regresión logística y ajustarlo a los datos de entrenamiento
lr_3= LogisticRegression(max_iter=1000,solver='newton-cg')
lr_3.fit(X_train, y_train)

# Hacer predicciones en los datos de prueba y calcular la precisión
y_pred_lr_3 = lr_3.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr_3)

print("Precisión de la regresión logística:", accuracy)
# Metricas de evaluacion
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_lr_3))
print("Recall: {}".format(recall_score(y_test, y_pred_lr_3, pos_label=1)))
print("Precisión: {}".format(precision_score(y_test, y_pred_lr_3, pos_label=1)))
print("Puntuación F1: {}".format(f1_score(y_test, y_pred_lr_3, pos_label=1)))

In [ ]:
# Matriz de confusion
cm_lr_3 = confusion_matrix(y_test, y_pred_lr_3)

# Crear una instancia de ConfusionMatrixDisplay con la matriz de confusión
cm_lr_3_display = ConfusionMatrixDisplay(confusion_matrix=cm_lr_3)

# Mostrar la matriz de confusión utilizando la función plot() de ConfusionMatrixDisplay
cm_lr_3_display.plot(cmap='Reds')

# Añadir etiquetas y títulos
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Mostrar la matriz de confusión
plt.show()

### OpenAI fine tuned classification
Se intenta utilizar un algoritmo usando tecnologia de punta, no se logra ka total implementacion debido a que tarda mucgo tiempo

In [ ]:
!pip install --upgrade openai

In [ ]:
import openai

In [ ]:
df_copy = df_reviews.copy()
df_copy['sentimiento'].replace({0 : 'negativo', 1 : 'positivo'}, inplace=True)
df_copy.rename(columns={'review_es': 'prompt', 'sentimiento': 'completion'}, inplace=True)
df_copy.head(5)

In [ ]:
df_copy.to_json("movies.jsonl", orient='records', lines=True)

In [ ]:
!pip install --upgrade openai
!openai tools fine_tunes.prepare_data -f movies.jsonl -q

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your key"

In [ ]:
!openai api fine_tunes.create -t "movies_prepared_train.jsonl" -m ada

## Resultados
Se prueba el modelo con el conjunto de datos sin clasificar
Primero se preparan estos datos, luego se carga el modelo, se realizan las predicciones y se evaluan

In [ ]:
df_reviews_new = pd.read_csv('./data/MovieReviewsPruebas.csv')
# Muestra el numero de filas y columnas del dataframe
print("tamano set pruebas: ",df_reviews_new.shape)
print(df_reviews.head(5))
# Muestra el tipo de datos de cada columna
print(df_reviews_new.dtypes)
# Muestra la cantidad de valores nulos para la columna review_es
print("valores nulos de review: ", df_reviews_new['review_es'].isnull().sum())
print("valores duplicados de review: ", df_reviews['review_es'].duplicated().sum())

# Primero creamos el diccionario de los idiomas en los que estan los comentarios
langs2 = {}
# Luego iteramos sobre cada comentario y determinamos su idioma
for index, row in df_reviews_new.iterrows():
    if detect(row['review_es']) not in langs2:
        langs2[detect(row['review_es'])] = 1
    else:
        langs2[detect(row['review_es'])] += 1

print(langs)

In [ ]:
# Eliminar repetidos
df_reviews_new['review_es'].drop_duplicates(inplace=True)
#Eliminar comentarios en ingles: Iteramos en cada una de las filas del dataframe
for index, row in df_reviews_new.iterrows():
    # Si el valor de la fila es en la elimina
    if detect(row['review_es']) != 'es':
        df_reviews_new.drop(index, inplace=True)

# Seteamos las stopwords en español
df_reviews_new['review_es'] = df_reviews_new['review_es'].apply(lambda x: ' '.join([snowball_stemmer.stem(word.lower()) for word in wordpunct_tokenize(x) if (word.isalpha() and word.lower() not in stop_words)]))
df_reviews_new['review_es'].head(5)

# Usamos el CountVectorizer() que nos da la cuenta de cada de cada palabra para cada review, lo cual podemos usar para los modelos que crearemos a continuación.
# Extraer la cuenta de cada palabra para cada review
X_new = vectorizer.transform(df_reviews_new['review_es'])


In [ ]:
#y_pred_lr_3_new = lr_3.predict(X_new)


# Exportación del modelo
Ahora realizaremos los pasos correspondientes a la exportación del mejor modelo para desplegarlo al púbico. Por lo que usaremos pipelines para completar esta tarea. De esta forma, primero debemos exportar el mejor modelo. Dados los resultados obtenidos, nos dimos cuenta que la diferencia entre la regresión logística y Random forest (los cuales obtenían sultados similares) se daba en la clasificación de falsos positivos y falsos negativos, por lo que consideramos usar la regresión logística como mejor modelo ya que para el negocio es mejor dejar una película no tan querida por el público a sacar del catálogo una película que si gusta.
Ahora, el pipeline debe ser de la siguiente forma:
- Eliminar los comentarios que están en ingles.
- Eliminar los duplicados
- Eliminar nulos
- Cambiar el string de la columna de "Sentimiento" de positivo y negativo por 0 y 1
- Hacemos la tokenización de las palabras de los reviews
- Vectorizamos
- Aplicamos el modelo
Una vez se haya hecho el pipeline, entrenaremos nuevamente el modelo y lo exportaremos usando la librería pickle

In [4]:
# Hacemos los imports necesarios para que funcione el modelo
import joblib as jb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Funcion para eliminar los comentarios que no esten en es
def drop_lang(x):
    for index, row in x.iterrows():
        if detect(row['review_es']) != 'es':
            x.drop(index, inplace=True)
    return x

# Funcion para cambiar los valores de la columna sentimiento
def change_sentiment(x):
    x['sentimiento'].replace({'negativo' : 0, 'positivo' : 1}, inplace=True)
    return x

# Funcion para eliminar los duplicados y los nulos
def dropper(x):
    x = x.drop(["Unnamed: 0"], axis=1) # Assign the modified DataFrame to x
    x['review_es'].drop_duplicates(inplace=True)
    x['review_es'].dropna(inplace=True)
    return x

Una vez definimos las funciones que nos ayudaran en la transformación de los comentarios recibidos, procedemos a crear el pipeline del modelo, entrenarlo y probarlo para posteriormente exportarlo.

In [5]:
vectorizer = jb.load('./data/count_vectorizer.pkl')
preprocessing = Pipeline([
    ('drop_lang', FunctionTransformer(drop_lang)),
    ('dropper', FunctionTransformer(dropper)),
    ('change_sentiment', FunctionTransformer(change_sentiment)),
])

model = Pipeline([
    ('clean_text', cleanText()),
    ('Vectorizer', vectorizer),
    ('Model', LogisticRegression(max_iter=1000,solver='newton-cg'))
])

# Definimos las variables que usaremos para entrenar y probar el modelo, estas deben ser los datos sin preprocesar
df_reviews_model = pd.read_csv('./data/MovieReviews.csv')

# Limpiamos los datos
df_reviews_model = preprocessing.fit_transform(df_reviews_model)

In [10]:
X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(df_reviews_model["review_es"], df_reviews_model['sentimiento'], test_size=0.3, random_state=42)
# Entrenamos el modelo
modelo_1 = model.fit(X_train_model, y_train_model)

In [5]:
# Ahora prombamos el modelo
y_model_pred = model.predict(X_test_model)
accuracy = accuracy_score(y_test_model, y_model_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8272033310201249


Con lo anterior pudimos comprobar que el el pipeline funiciona correctamente por lo que ahora realizaremos el entrenamiento con todos los datos limpios. y exportaremos el modelo final

In [6]:
import pickle
# Guardamos el modelo
modelo_final = model.fit(df_reviews_model["review_es"], df_reviews_model['sentimiento'])
# pickle.dump(modelo_final, open('./data/prediccion_reviews.pkl', 'wb'))

In [20]:
print(X_test_model.head(5))

617     Me sorprendió las mejoras realizadas en una pe...
3509    Esta película sigue un guión gráfico muy simil...
2816    Un molesto y sin talento, el creador documenta...
2578    Finalmente he visto el incubus después de espe...
8       Asesina sexy Tiffany (Jennifer Tilly) todavía ...
Name: review_es, dtype: object


Por ultimo debemos exportar el modelo. Debido a fallos con FastAPI el modelo se debe exportar desde un archivo de python aparte, el cual se puede encontrar en la carpeta de API bajo el nombre de model_pipeline.py